In [1]:
import numpy as np
import pandas as pd
from main import tree_grow, tree_grow_b, tree_pred, tree_pred_b
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from IPython.display import display, Markdown, Latex
from mlxtend.evaluate import mcnemar
from mlxtend.evaluate import mcnemar_table
from time import time

pd.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv("eclipse-metrics-packages-2.0.csv", delimiter=";")
test = pd.read_csv("eclipse-metrics-packages-3.0.csv", delimiter=";")
list(train.columns[4:44])
feats = ['pre',
 'ACD_avg',
 'ACD_max',
 'ACD_sum',
 'FOUT_avg',
 'FOUT_max',
 'FOUT_sum',
 'MLOC_avg',
 'MLOC_max',
 'MLOC_sum',
 'NBD_avg',
 'NBD_max',
 'NBD_sum',
 'NOCU',
 'NOF_avg',
 'NOF_max',
 'NOF_sum',
 'NOI_avg',
 'NOI_max',
 'NOI_sum',
 'NOM_avg',
 'NOM_max',
 'NOM_sum',
 'NOT_avg',
 'NOT_max',
 'NOT_sum',
 'NSF_avg',
 'NSF_max',
 'NSF_sum',
 'NSM_avg',
 'NSM_max',
 'NSM_sum',
 'PAR_avg',
 'PAR_max',
 'PAR_sum',
 'TLOC_avg',
 'TLOC_max',
 'TLOC_sum',
 'VG_avg',
 'VG_max',
 'VG_sum']
train_x = train[feats]
train_y = train["post"]
test_x = test[feats]
test_y = test["post"]

In [3]:
train_x.describe()

,pre,ACD_avg,ACD_max,ACD_sum,FOUT_avg,FOUT_max,FOUT_sum,MLOC_avg,MLOC_max,MLOC_sum,NBD_avg,NBD_max,NBD_sum,NOCU,NOF_avg,NOF_max,NOF_sum,NOI_avg,NOI_max,NOI_sum,NOM_avg,NOM_max,NOM_sum,NOT_avg,NOT_max,NOT_sum,NSF_avg,NSF_max,NSF_sum,NSM_avg,NSM_max,NSM_sum,PAR_avg,PAR_max,PAR_sum,TLOC_avg,TLOC_max,TLOC_sum,VG_avg,VG_max,VG_sum
count,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000
mean,11.397878,0.487768,2.639257,7.559682,3.569337,37.143236,726.572944,6.886520,72.989390,1421.830239,1.408035,4.970822,284.503979,17.848806,2.655245,12.607427,54.798408,0.172434,0.509284,2.822281,7.919620,32.379310,177.769231,0.827946,0.978780,15.034483,2.277422,21.811671,41.076923,1.181801,12.604775,21.246684,1.020060,4.957560,210.114058,117.645997,494.687003,2113.901857,2.276089,19.809019,480.114058
std,21.791585,0.914659,3.739858,16.867357,2.278178,36.675557,1722.374379,4.667409,90.803798,3634.398730,0.537000,2.323950,687.157223,22.557750,3.885536,20.484679,94.776931,0.277450,0.500578,6.726633,5.384483,37.169636,443.802545,0.277751,0.177379,21.431146,6.283659,72.670226,94.109527,3.402363,51.386834,69.903088,0.439893,3.203771,455.568578,127.328041,593.967554,4824.224327,1.327293,29.697421,1343.046313
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.800000,6.000000,6.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,2.166667,16.000000,117.000000,4.405405,29.000000,217.000000,1.233333,4.000000,51.000000,6.000000,1.000000,4.000000,10.000000,0.000000,0.000000,0.000000,4.842105,13.000000,37.000000,0.785714,1.000000,3.000000,0.383333,2.000000,4.000000,0.037037,1.000000,1.000000,0.743455,3.000000,44.000000,59.700000,176.000000,421.000000,1.673913,7.000000,71.000000
50%,3.000000,0.148148,1.000000,1.000000,3.521277,29.000000,331.000000,6.627737,54.000000,677.000000,1.468227,5.000000,143.000000,11.000000,2.076923,8.000000,25.000000,0.019048,1.000000,1.000000,6.780488,22.000000,83.000000,0.980952,1.000000,10.000000,1.000000,6.000000,12.000000,0.416667,3.000000,6.000000,0.954220,4.000000,105.000000,97.153846,327.000000,1080.000000,2.211268,13.000000,228.000000
75%,12.000000,0.620690,4.000000,8.000000,4.830189,45.000000,854.000000,8.674699,88.000000,1529.000000,1.750000,6.000000,320.000000,22.000000,3.352941,14.000000,62.000000,0.214286,1.000000,3.000000,9.500000,40.000000,209.000000,1.000000,1.000000,18.000000,2.000000,14.000000,33.000000,1.090909,10.000000,18.000000,1.173077,6.000000,224.000000,140.000000,595.000000,2305.000000,2.674699,22.000000,535.000000
max,179.000000,9.000000,24.000000,150.000000,20.500000,310.000000,28201.000000,39.500000,994.000000,60190.000000,3.000000,17.000000,11611.000000,242.000000,63.250000,251.000000,994.000000,1.000000,1.000000,76.000000,42.000000,297.000000,7669.000000,1.111111,2.000000,241.000000,94.333333,1049.000000,1119.000000,56.714286,596.000000,938.000000,3.000000,28.000000,7059.000000,1872.000000,5207.000000,79474.000000,12.460000,310.000000,23255.000000


In [4]:
train_y.describe()

count    377.000000
mean       2.432361
std        6.241702
min        0.000000
25%        0.000000
50%        1.000000
75%        3.000000
max       88.000000
Name: post, dtype: float64

In [5]:
train_x_np = train_x.to_numpy()
test_x_np = test_x.to_numpy()

In [6]:
train_y_np = train_y.to_numpy()
train_y_np = np.where(train_y_np > 0, 1, 0)
test_y_np = test_y.to_numpy()
test_y_np = np.where(test_y_np > 0, 1, 0)

In [7]:
preds_model = {}

In [8]:
start = time()
single_tree = tree_grow(train_x_np, train_y_np, 15, 5, 41)
end = time()
print(f"Training single tree took {end-start} seconds")
test_y_np_pred_st = tree_pred(test_x_np, single_tree)
preds_model["single tree"] = test_y_np_pred_st

Training single tree took 1.0782661437988281 seconds


In [9]:
# len(single_tree.y) == (len(single_tree.l.y) + len(single_tree.r.y))

In [10]:
def report_scores(y_true,y_pred):
    conf_mat = confusion_matrix(y_true,y_pred)
    tn = conf_mat[0][0]
    fp = conf_mat[0][1]
    fn = conf_mat[1][0]
    tp = conf_mat[1][1]
    print("Confusion matrix:")
    display(Markdown(f'''| True/Pred |Pos|Neg|
|-----------|---|---|
|       Pos |{tp}|{fn}|
|       Neg |{fp}|{tn}|
    '''))
    print("---------------------------------")
    print("LaTex formatting:", '''
\\begin{table}[]
\\begin{tabular}{|r|l|l|}
\\hline
True/Pred & Pos & Neg \\\ \hline''')
    print(f"Pos                             &  {tp}   &  {fn}   \\\ \hline")
    print(f"Neg                             & {fp}    &  {tn}   \\\ \hline")
    print('''\end{tabular}
\end{table}''')
    print("------------------------------------")
    print("accuracy:", accuracy_score(y_true,y_pred))
    print("precision:", precision_score(y_true,y_pred))
    print("recall:", recall_score(y_true,y_pred))

In [11]:
report_scores(test_y_np, test_y_np_pred_st)

Confusion matrix:


| True/Pred |Pos|Neg|
|-----------|---|---|
|       Pos |185|128|
|       Neg |82|266|
    

---------------------------------
LaTex formatting: 
\begin{table}[]
\begin{tabular}{|r|l|l|}
\hline
True/Pred & Pos & Neg \\ \hline
Pos                             &  185   &  128   \\ \hline
Neg                             & 82    &  266   \\ \hline
\end{tabular}
\end{table}
------------------------------------
accuracy: 0.6822995461422088
precision: 0.6928838951310862
recall: 0.5910543130990416


In [12]:
# Print the first two-layers of the tree
# Root node information
root = single_tree
split_feature = feats[root.f]
split_value = root.s
y_values = root.y
y_values_1_ratio = sum(y_values)/len(y_values)
print(f"Root values: split_feature = {split_feature}, split_value = {split_value}, y_values_1_ratio = {y_values_1_ratio}")

Root values: split_feature = pre, split_value = 4.5, y_values_1_ratio = 0.5039787798408488


In [13]:
print(np.bincount(single_tree.y))
# print(single_tree.f)
print(single_tree.s)
print(feats[single_tree.f])

[187 190]
4.5
pre


In [14]:
print(np.bincount(single_tree.l.y))
print(single_tree.l.s)
print(feats[single_tree.l.f])

[156  58]
26.5
VG_max


In [15]:
print(np.bincount(single_tree.r.y))
print(single_tree.r.s)
print(feats[single_tree.r.f])

[ 31 132]
0.1583333333333333
NOI_avg


In [16]:
print(np.bincount(single_tree.l.l.y))
print(single_tree.l.l.s)
print(feats[single_tree.l.l.f])

[150  41]
54.5
NSM_sum


In [17]:
print(np.bincount(single_tree.l.r.y))
print(single_tree.l.r.s)
print(feats[single_tree.l.r.f])

[ 6 17]
358.0
VG_sum


In [18]:
print(np.bincount(single_tree.r.l.y))
print(single_tree.r.l.s)
print(feats[single_tree.r.l.f])

[ 11 107]
11.5
pre


In [19]:
print(np.bincount(single_tree.r.r.y))
print(single_tree.r.r.s)
print(feats[single_tree.r.r.f])

[20 25]
9.5
NOCU


In [ ]:
start = time()
bagging_tree = tree_grow_b(train_x_np, train_y_np, 15, 5, 41, 100)
end = time()
print(f"Training bagging tree took {end-start} seconds")
test_y_np_pred_bt = tree_pred_b(test_x_np, bagging_tree)
preds_model["bagging tree"] = test_y_np_pred_bt

In [ ]:
report_scores(test_y_np, test_y_np_pred_bt)

In [ ]:
start = time()
random_tree = tree_grow_b(train_x_np, train_y_np, 15, 5, 6, 100)
end = time()
print(f"Training random forest took {end-start} seconds")
test_y_np_pred_rt = tree_pred_b(test_x_np, random_tree)
preds_model["random forest"] = test_y_np_pred_rt

In [ ]:
report_scores(test_y_np, test_y_np_pred_rt)

In [ ]:
models = list(preds_model.keys())
tables = []
for model1 in models:
    for model2 in models:
        if not(model1 == model2):
            print(f"McNemar's test {model1} vs {model2}")
            # The code in the following three lines was adopted from [1].
            table = mcnemar_table(y_target=test_y_np, y_model1=preds_model[model1], y_model2=preds_model[model2])
            tables.append(table)
            chi2_, p = mcnemar(ary=table, corrected=True)
            print(f"chi² statistic: {chi2_}, p-value: {p}\n")
    models.remove(model1)

In [ ]:
for table in tables:
    print("LaTex formatting:", '''
    \\begin{table}[]
    \\begin{tabular}{|r|l|l|}
    \\hline
    Model1/Model2 & Wrong & Right \\\ \hline''')
    print(f"Wrong                             &  {table[0][0]}   &  {table[0][1]}   \\\ \hline")
    print(f"Right                             & {table[1][0]}    &  {table[1][1]}   \\\ \hline")
    print('''\end{tabular}
    \end{table}''')
    print("------------------------------")

# References
[1] T. Toledo Jr, “Statistical tests for comparing classification algorithms,” Medium, Jan. 04, 2022. [Online]. Available: https://towardsdatascience.com/statistical-tests-for-comparing-classification-algorithms-ac1804e79bb7